承知いたしました。素晴らしいですね！ここからは再び学習フェーズとして、**Pythonファイル**と**Jupyter Notebook形式**の2本立てで、一歩ずつ量子アルゴリズムの世界を探求していきましょう。

-----

## 応用編 - ステップ1: n量子ビットCNOTゲートの実装

量子アルゴリズムを実装する上で、複数の量子ビットをまたいで操作する**制御ゲート**、特に**CNOTゲート**は不可欠です。しかし、これまでのシミュレータは1量子ビットゲートにしか対応していませんでした。

そこで最初のステップとして、シミュレータエンジンをアップグレードし、任意の量子ビット数 `n` において、任意のコントロールビットとターゲットビットを指定できるCNOTゲートを実装します。

-----

-----

### 1\. Pythonファイル (`quantum_simulator_cnot_engine.py`)

まずは、`QuantumSimulator` クラスに `apply_cnot` メソッドを追加し、その動作をターミナルで確認するためのスクリプトです。

```python
# quantum_simulator_cnot_engine.py

import numpy as np
from quantum_simulator import gates # フェーズ1で作成したgates.pyを使用

class QuantumSimulator:
    """CNOTゲートに対応したn量子ビットシミュレータ"""
    def __init__(self, n_qubits):
        if n_qubits <= 0:
            raise ValueError("量子ビット数は1以上である必要があります。")
        self.n_qubits = n_qubits
        self.state_vector = np.zeros(2**n_qubits, dtype=complex)
        self.state_vector[0] = 1

    def _get_operator(self, gate, target_qubit):
        op_list = [gates.I] * self.n_qubits
        op_list[target_qubit] = gate
        operator = op_list[0]
        for i in range(1, self.n_qubits):
            operator = np.kron(operator, op_list[i])
        return operator

    def apply_gate(self, gate, target_qubit):
        """1量子ビットゲートを適用"""
        operator = self._get_operator(gate, target_qubit)
        self.state_vector = np.dot(operator, self.state_vector)

    def apply_cnot(self, control, target):
        """
        n量子ビットの状態にCNOTゲートを適用する
        :param control: コントロール量子ビットのインデックス
        :param target: ターゲット量子ビットのインデックス
        """
        if control == target or not (0 <= control < self.n_qubits and 0 <= target < self.n_qubits):
            raise ValueError("CNOTのコントロール、ターゲット指定が無効です。")

        # CNOTは特定の基底状態の確率振幅を入れ替える操作
        for i in range(2**self.n_qubits):
            # 基底状態を2進数文字列で表現
            basis = format(i, f'0{self.n_qubits}b')
            
            # コントロールビットが'1'の場合のみ操作を考える
            if basis[control] == '1':
                # ターゲットビットを反転させた状態のインデックスjを計算
                target_flipped_basis = list(basis)
                target_flipped_basis[target] = '1' if basis[target] == '0' else '0'
                j = int("".join(target_flipped_basis), 2)
                
                # i < j の場合のみ入れ替える（二重処理を防ぐため）
                if i < j:
                    # 振幅をスワップ
                    self.state_vector[i], self.state_vector[j] = self.state_vector[j], self.state_vector[i]
    
    def measure(self, shots=1):
        probabilities = np.abs(self.state_vector)**2
        probabilities /= np.sum(probabilities)
        outcomes = np.arange(2**self.n_qubits)
        results_dec = np.random.choice(outcomes, size=shots, p=probabilities)
        return [format(res, f'0{self.n_qubits}b') for res in results_dec]

# --- 動作確認 ---
if __name__ == "__main__":
    # 3量子ビットでGHZ状態 (エンタングルメント状態) を作る
    n = 3
    sim = QuantumSimulator(n)

    print(f"--- {n}量子ビットGHZ状態の生成 ---")
    print(f"初期状態: |000>")
    
    # 1. q0にHゲートを適用 -> 1/√2 * (|000> + |100>)
    sim.apply_gate(gates.H, 0)

    # 2. CNOT(control=0, target=1) を適用 -> 1/√2 * (|000> + |110>)
    sim.apply_cnot(0, 1)

    # 3. CNOT(control=0, target=2) を適用 -> 1/√2 * (|000> + |111>)
    sim.apply_cnot(0, 2)

    print("\n最終的な状態ベクトル:")
    for i, amp in enumerate(sim.state_vector):
        if not np.isclose(amp, 0):
            print(f"  振幅: {amp:.4f}, 基底: |{format(i, f'0{n}b')}>")
    
    print("\n期待されるGHZ状態は |000> と |111> の重ね合わせです。")
```

-----

-----

### 2\. Jupyter Notebook形式

次に、Jupyter Notebookで解説と共に実行できる形式です。

### [Markdownセル]

# 応用編 - ステップ1: n量子ビットCNOTゲートの実装 🛠️

本格的な量子アルゴリズムを実装するためには、複数の量子ビットを操作する**CNOTゲート**が不可欠です。このノートブックでは、シミュレータのエンジンをアップグレードし、任意の量子ビット数 `n` において、好きな量子ビットをコントロール、ターゲットに指定できるCNOTゲートを実装します。

## 1\. CNOTゲートの仕組み (n量子ビット)

n量子ビット系において、CNOTゲートは**状態ベクトル内の確率振幅を入れ替える**操作として考えることができます。

ルールはシンプルです。
**「コントロールビットが `1` であるすべての基底状態について、ターゲットビットの `0` と `1` に対応する振幅のペアを入れ替える」**

例えば、3量子ビット系で `CNOT(control=0, target=2)` を適用する場合：

  - コントロールビット `q0` が `1` の基底状態は `|100>, |101>, |110>, |111>` です。
  - これらのうち、ターゲットビット `q2` が異なるペアを探します。
      - `|100>` (4) と `|101>` (5) の振幅を入れ替える
      - `|110>` (6) と `|111>` (7) の振幅を入れ替える

このロジックをシミュレータに実装します。

-----

### [Codeセル]

```python
import numpy as np
# フェーズ1で作成したgates.pyが必要です
from quantum_simulator import gates

class QuantumSimulator:
    """CNOTゲートに対応したn量子ビットシミュレータ"""
    def __init__(self, n_qubits):
        if n_qubits <= 0:
            raise ValueError("量子ビット数は1以上である必要があります。")
        self.n_qubits = n_qubits
        self.state_vector = np.zeros(2**n_qubits, dtype=complex)
        self.state_vector[0] = 1

    def apply_gate(self, gate, target_qubit):
        """1量子ビットゲートを適用"""
        op_list = [gates.I] * self.n_qubits
        op_list[target_qubit] = gate
        operator = op_list[0]
        for i in range(1, self.n_qubits):
            operator = np.kron(operator, op_list[i])
        self.state_vector = np.dot(operator, self.state_vector)

    def apply_cnot(self, control, target):
        """n量子ビットの状態にCNOTゲートを適用するメソッド"""
        if control == target or not (0 <= control < self.n_qubits and 0 <= target < self.n_qubits):
            raise ValueError("CNOTのコントロール、ターゲット指定が無効です。")
        
        # 状態ベクトルの全要素をチェック
        for i in range(2**self.n_qubits):
            basis = format(i, f'0{self.n_qubits}b')
            
            # コントロールビットが'1'か？
            if basis[control] == '1':
                # '1'なら、ターゲットビットを反転させた相方のインデックスjを計算
                target_flipped_basis = list(basis)
                target_flipped_basis[target] = '1' if basis[target] == '0' else '0'
                j = int("".join(target_flipped_basis), 2)
                
                # 同じペアを2回スワップしないように、i < j の場合のみ処理
                if i < j:
                    # 振幅を入れ替える
                    self.state_vector[i], self.state_vector[j] = self.state_vector[j], self.state_vector[i]
    
    def measure(self, shots=1):
        probabilities = np.abs(self.state_vector)**2
        probabilities /= np.sum(probabilities)
        outcomes = np.arange(2**self.n_qubits)
        results_dec = np.random.choice(outcomes, size=shots, p=probabilities)
        return [format(res, f'0{self.n_qubits}b') for res in results_dec]

```

-----

### [Markdownセル]

## 2\. 動作確認：GHZ状態の生成

新しい `apply_cnot` メソッドが正しく動作するか、3量子ビットの有名なエンタングルメント状態である**GHZ状態** `1/√2 * (|000> + |111>)` を作って確認してみましょう。

**生成手順:**

1.  初期状態 `|000>`
2.  `q0` にHゲートを適用
3.  `CNOT(control=0, target=1)` を適用
4.  `CNOT(control=0, target=2)` を適用

-----

### [Codeセル]

```python
n = 3
sim = QuantumSimulator(n)
print(f"--- {n}量子ビットGHZ状態の生成 ---")
print(f"初期状態: |000>")

# ステップ1: H on q0
sim.apply_gate(gates.H, 0)
# ステップ2: CNOT(0, 1)
sim.apply_cnot(0, 1)
# ステップ3: CNOT(0, 2)
sim.apply_cnot(0, 2)

print("\n最終的な状態ベクトル:")
final_state_amps = []
for i, amp in enumerate(sim.state_vector):
    if not np.isclose(amp, 0):
        final_state_amps.append(f"  振幅: {amp:.4f}, 基底: |{format(i, f'0{n}b')}>")
print("\n".join(final_state_amps))

# 検算
expected_amps = {0: 1/np.sqrt(2), 7: 1/np.sqrt(2)} # |000>と|111>
correct = True
for i, amp in enumerate(sim.state_vector):
    if i in expected_amps:
        if not np.isclose(amp, expected_amps[i]):
            correct = False
    elif not np.isclose(amp, 0):
        correct = False
print(f"\n結果は期待通りですか？: {correct}")
```

-----

### [Markdownセル]

これで、シミュレータのエンジンはCNOTゲートに対応できました。
次のステップでは、この新しいエンジンを使って、**Deutsch-Jozsaアルゴリズム**をGUIから実行できるようにプリセット機能を追加していきます。